In [1]:
# Import library yang diperlukan
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error


In [2]:
# Baca dataset dari file CSV
df = pd.read_csv('DATA_RUMAH.csv')

# Menampilkan 5 baris teratas
df.head(5)

,NO,NAMA RUMAH,HARGA,LB,LT,KT,KM,GRS
0,1,"Rumah Murah Hook Tebet Timur, Tebet, Jakarta S...",3800000000,220,220,3,3,0
1,2,"Rumah Modern di Tebet dekat Stasiun, Tebet, Ja...",4600000000,180,137,4,3,2
2,3,"Rumah Mewah 2 Lantai Hanya 3 Menit Ke Tebet, T...",3000000000,267,250,4,4,4
3,4,"Rumah Baru Tebet, Tebet, Jakarta Selatan",430000000,40,25,2,2,0
4,5,"Rumah Bagus Tebet komp Gudang Peluru lt 350m, ...",9000000000,400,355,6,5,3


In [3]:
# Menampilkan informasi dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NO          1010 non-null   int64 
 1   NAMA RUMAH  1010 non-null   object
 2   HARGA       1010 non-null   int64 
 3   LB          1010 non-null   int64 
 4   LT          1010 non-null   int64 
 5   KT          1010 non-null   int64 
 6   KM          1010 non-null   int64 
 7   GRS         1010 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 63.3+ KB


In [4]:
# Mengecek apakah ada nilai kosong di dataset
print(df.isnull().sum())

NO            0
NAMA RUMAH    0
HARGA         0
LB            0
LT            0
KT            0
KM            0
GRS           0
dtype: int64


In [5]:
# check duplicate
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1005    False
1006    False
1007    False
1008    False
1009    False
Length: 1010, dtype: bool

In [20]:
# Menghapus kolom "NO" dan "NAMA RUMAH" 
df_cleaned = df.drop(columns=["NO", "NAMA RUMAH"])
df_cleaned

,HARGA,LB,LT,KT,KM,GRS
0,3800000000,220,220,3,3,0
1,4600000000,180,137,4,3,2
2,3000000000,267,250,4,4,4
3,430000000,40,25,2,2,0
4,9000000000,400,355,6,5,3
...,...,...,...,...,...,...
1005,9000000000,450,550,10,10,3
1006,4000000000,160,140,4,3,2
1007,4000000000,139,230,4,4,1
1008,19000000000,360,606,7,4,0


In [21]:
# Menentukan fitur dan target dari dataset yang sudah dibersihkan
X = df_cleaned[["LB", "LT", "KT", "KM", "GRS"]]  # Fitur
y = df_cleaned["HARGA"]  # Target

In [22]:
# Membagi dataset menjadi training set dan testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [29]:
# Membuat model Random Forest Regressor
rf = RandomForestRegressor(random_state=42)

In [30]:
# Melatih model dengan data latih
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:
# Tuning Hyperparameter dengan GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Jumlah pohon (trees) yang akan digunakan dalam Random Forest
    'max_features': ['sqrt', 'log2'], # Opsi fitur yang digunakan untuk membagi setiap node (sqrt atau log2), 'sqrt' (akar kuadrat dari total fitur) 'log2' (logaritma base 2 dari total fitur).
    'max_depth': [None, 10, 20, 30], # Kedalaman maksimal pohon, None = tanpa batas
    'min_samples_split': [2, 5, 10] # Jumlah minimal sampel untuk membagi sebuah node
}

In [32]:
# Menginisialisasi GridSearchCV dengan parameter yang telah ditentukan
# Estimator adalah model Random Forest, param_grid adalah parameter yang akan dicari, cv=3 untuk cross-validation 3-fold
# n_jobs=-1 akan menggunakan semua inti prosesor untuk mempercepat pencarian
# verbose=2 memberikan informasi lebih rinci tentang proses pencarian
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

In [33]:
# GridSearchCV akan mencoba setiap kombinasi parameter dan memilih yang terbaik
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             verbose=2)

In [34]:
# Tampilkan parameter terbaik
print(f"Best parameters: {grid_search.best_params_}")

Best parameters: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}


In [35]:
# Membuat prediksi pada data uji menggunakan model terbaik
best_rf = grid_search.best_estimator_  # Menggunakan model terbaik dari GridSearchCV
y_pred_rf = best_rf.predict(X_test)

In [36]:
# Menghitung R-Squared untuk mengevaluasi model
r2_rf = r2_score(y_test, y_pred_rf)
print(f"R Squared = {r2_rf}")

R Squared = 0.8234138005080476


In [38]:
# Save Model
import joblib

# Save model ke file 
joblib.dump(rf, 'house_price_ranfor_model.pkl')

['house_price_ranfor_model.pkl']